# Decorator

A decorator is a callable that takes another function as argument(the decorated function).

This code,

```python
@decorate
def target():
    print("running target()")
```
Has the same effect as writing this:
```python
def target():
    print("running target()")

target = decorate(target)
```



In [1]:
def decorate(func):
    def inner():
        print("running inner()")
    return inner

@decorate
def target():
    print("running target()")

In [2]:
target()

running inner()


## When Python executes decorators

Decorators are executed as soon as the module is imported, but the decorator function only run when they are explicitly invoked.

In [3]:
registry=[]

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print("running f1()")
    
@register
def f2():
    print("running f2()")

running register <function f1 at 0x110b8d9d8>
running register <function f2 at 0x110b8dea0>


In [4]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In the above example, even though the register decorator returns the decorated function unchanged, it is not useless. The similar techniques are used in many Python web frameworks to add functions to some central registry, for example, a registry mapping URL patterns to functions that generate HTTP responses.

## Decorator-enhanced Strategy pattern

In [5]:
promos = []

def promotion(promo_fun):
    promos.append(promo_fun)
    return promo_fun

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total()*.05 if order.customer.fidelity >=1000 else 0

@promotion
def bulk_item(order):
    """10% discount for orders with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quatity >=10:
            discount+=item.total()*0.1
    return discount

@promotion
def large_order(order):
    """7% discount for all orders with 10 or more disticnt items"""
    distinct_items = {item.product for item in order.cart}
    
    if len(distinct_items)>=10:
        return order.total*.07
    return 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

Advantages of this approach:
 * The @promotion highlights the purpose of the decorated function, and also makes it easy to temporarily disable a promotion; just comment out the decorator.
 * Promotional discount strategies may be defined in other modules, anywhere in the system, as long as the @promotion decorator is applied to them.

## Variable scope rules

In [12]:
b = 6
def f2(a):
    print(a)
    print(b)

In [13]:
f2(1)

1
6


What if we include, an assigment statement for b after printing it?

In [14]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

In [17]:
import sys
try:
    f2(4)
except Exception as e:
    print(e, file=sys.stderr)

4


local variable 'b' referenced before assignment


The fact is, when Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function.

_This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local._

If we want the interpreter to treat b as a global variable in spite of the assignment within the function, we use `global` declaration.

In [18]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b=9

In [19]:
f2(4)

4
6


In [20]:
b

9

In [21]:
from dis import dis

In [22]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


The CPython VM that runs the bytecode is a stack machine, so the operations LOAD and POP refer to the stack.

## Closures

Actually, a closure is a function with an extended scope that encompasses non-global variables referenced in the body of the function but not defined there. _It does not matter whether the function is anonymous or not, what matters is that it can access non-global variables that are defined outside of the body._

A class to calculate running average:

In [23]:
class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        return sum(self.series)/len(self.series)

In [25]:
avg = Averager()
avg(10)

10.0

In [26]:
avg(11)

10.5

Below is a functional implementation for the same:

In [29]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        
        return sum(series)/len(series)
    return averager

<img src="images/closure1.png" width="250"/>
Within averager, series is a free variable. This is a technical term meaning a variable that is bound in the local scope.

In [30]:
avg = make_averager()
avg(10)

10.0

In [31]:
avg.__code__.co_varnames

('new_value',)

In [32]:
avg.__code__.co_freevars

('series',)

The only situation in which a function may need to deal with external variables that are non-global is when it is nested in another function.

## The nonlocal declaration

The below code tries to make the make_averager better, by avoiding the storage for all the inputs used.

In [41]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count+=1
        total+=new_value
        
        return total/count
    return averager

In [42]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

The problem is that the statement count += 1 actually means the same as count = count + 1, when count is a number or any immutable type. So we are actually assigning to count in the body of averager, and that makes it a local variable.

To workaround this, the `nonlocal` declaration was introduced in Python 3. It lets you flag a variable as a free variable even when it is assigned a new value within the function. 

In [43]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count+=1
        total+=new_value
        
        return total/count
    return averager

In [44]:
avg = make_averager()
avg(10)

10.0

## Implementing a single decorator

A simple decorator to output the running time of functions

In [46]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter()-t0
        
        name = func.__name__
        arg_str = ','.join(repr(arg) for arg in args)
        
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result ))
        return result
    return clocked

In [49]:
from functools import reduce
from operator import mul
@clock
def factorial(n):
    return reduce(mul, range(1, n+1))

In [50]:
[factorial(n) for n in range(1, 5)]

[0.00000235s] factorial(1) -> 1
[0.00000482s] factorial(2) -> 2
[0.00000265s] factorial(3) -> 6
[0.00000158s] factorial(4) -> 24


[1, 2, 6, 24]

In [51]:
factorial.__name__

'clocked'

Let's improve the clock implementation by supporting the keyword arguments, and copy relevant attributes from func to clocked.

In [54]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter()-t0
        
        arg_list=[]
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        
        if kwargs:
            pairs = [f'{k}={v}' for k,v in kwargs.items()]
            arg_list.append(', '.join(pairs))
        
        name = func.__name__
        arg_str = ','.join(arg_list)
        
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result ))
        return result
    return clocked

## Decorators in the standard library

### Memoization with functools.lru_cache

The very costly recursive way to compute the Nth number in the Fibonocci series.

In [56]:
@clock
def fibonacci(n):
    if n<2:
        return n
    
    return fibonacci(n-1)+fibonacci(n-2)

In [59]:
fibonacci(7)

[0.00000059s] fibonacci(1) -> 1
[0.00000121s] fibonacci(0) -> 0
[0.00021414s] fibonacci(2) -> 1
[0.00000052s] fibonacci(1) -> 1
[0.00027737s] fibonacci(3) -> 2
[0.00000041s] fibonacci(1) -> 1
[0.00000101s] fibonacci(0) -> 0
[0.00027358s] fibonacci(2) -> 1
[0.00069276s] fibonacci(4) -> 3
[0.00000038s] fibonacci(1) -> 1
[0.00000042s] fibonacci(0) -> 0
[0.00007576s] fibonacci(2) -> 1
[0.00000059s] fibonacci(1) -> 1
[0.00013292s] fibonacci(3) -> 2
[0.00088870s] fibonacci(5) -> 5
[0.00000033s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00004702s] fibonacci(2) -> 1
[0.00000032s] fibonacci(1) -> 1
[0.00008817s] fibonacci(3) -> 2
[0.00000030s] fibonacci(1) -> 1
[0.00000105s] fibonacci(0) -> 0
[0.00016157s] fibonacci(2) -> 1
[0.00045505s] fibonacci(4) -> 3
[0.00140791s] fibonacci(6) -> 8
[0.00000043s] fibonacci(1) -> 1
[0.00000041s] fibonacci(0) -> 0
[0.00005760s] fibonacci(2) -> 1
[0.00000040s] fibonacci(1) -> 1
[0.00012071s] fibonacci(3) -> 2
[0.00000031s] fibonacci(1) -> 1
[0.00000

13

Faster implementation using caching.

In [60]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    
    return fibonacci(n-1)+fibonacci(n-2)

In [61]:
fibonacci(7)

[0.00000065s] fibonacci(1) -> 1
[0.00000048s] fibonacci(0) -> 0
[0.00017859s] fibonacci(2) -> 1
[0.00021618s] fibonacci(3) -> 2
[0.00023932s] fibonacci(4) -> 3
[0.00028955s] fibonacci(5) -> 5
[0.00031572s] fibonacci(6) -> 8
[0.00034019s] fibonacci(7) -> 13


13

Besides making silly recursive algorithms viable, lru_cache really shines in applications that need to fetch information from web.

```python
functools.lru_cache(maxsize=128, typed=False)```

The typed argument, if set to True, stores results of different argument types separately, i.e. distinguishing between float and integer arguments that are normally considered equal, like 1 and 1.0.

_Since_ `lru_cache` _uses a dict to store the results, and the keys are made from positional and keyword arguments used in the calls, all the arguments taken by the decorated function must be hashable._ 

### Generic functions with single dispatch

Check Fluent Python - 203

## Stacked decorators

When two decorators `@d1` and `@d2` are applied to a function f in that order, the result is same as `f = d1(d2(f))`.

Means,

```python
@d1
@d2
def f():
    print("f")
```    

Is the same as:
```python
def f():
    print("f")

f = d1(d2(f))
```

## Parameterized Decorators

How do you make a decorator accept other arguments?
The answer is: make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated.

In [70]:
registry = set()

def register(active=True):
    def decorate(func):
        print(f'running register(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate

In [71]:
@register(active=False)
def f1():
    print("running f1()")

running register(active=False)->decorate(<function f1 at 0x110b8d8c8>)


The above deocorator fun is equivalent to:
`f = register(active=False)(f)`